In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from tensorflow.keras.models import load_model
from keras.datasets import mnist
import cv2
import numpy as np
from keras.utils import np_utils

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
def load_data():
  data_test = np.load('/content/drive/MyDrive/Colab Notebooks/data_train.npy')
  labels_test = np.load('/content/drive/MyDrive/Colab Notebooks/labels_train.npy')
  return data_test,labels_test

In [10]:
def preprocess_data(data_test):
  #reshaping the original  90,000 x 1 to a 300 x 300 then to 30 X 30 size.
  resize_image_size = 30
  resized_dataset = []#list to store the resized samples
  for i in range(data_test.shape[1]):# range of 90,000 per digit for 3360 digits
      single_image = data_test[:,i]# each image is made up of 90,000 pixels
      single_image = single_image.reshape(300,300)# reshaping the original image from 90,000 x 1 to a 300 x 300
      resizing_dim = (resize_image_size,resize_image_size)
      resized_image = cv2.resize(single_image, resizing_dim, interpolation = cv2.INTER_AREA)# For the interpolation, we use INTER_AREA: which resamples using pixel area relation. It may be a preferred method of image decimation, as it gives more free results. But when the image is zoomed out, it is similar to the INTER_NEAREST method
      resized_image = resized_image.reshape(resize_image_size*resize_image_size)
      resized_dataset.append(resized_image)
  resized_dataset_array = np.array(resized_dataset)
  #print(resized_dataset_array.shape)
  data_resized_test = np.array(resized_dataset).T

  #Morphological transformation of closing operation
  # closing of resized (30x30) dataset
  closing_resized_dataset = []#list to store the closing resized samples
  for i in range(data_resized_test.shape[1]): 
      single_image = data_resized_test[:,i] #taking each column -> each image
      kernel = np.ones((1,1),np.uint8) #kernal of size 1x1 pixel(optimum for resized dataset)
      closing_image = cv2.morphologyEx(single_image,cv2.MORPH_CLOSE,kernel)
      closing_resized_dataset.append(closing_image)
  closing_resized_dataset_array = np.array(closing_resized_dataset)
  data_closing = np.array(closing_resized_dataset).T
  data_closing= data_closing.squeeze()
  return data_closing
  

In [25]:
#function to provide the output of the predicted class of the test function
def evaluate(Input_test,labels): 
    im_siz =30
    test_set = Input_test.reshape(Input_test.shape[0], im_siz, im_siz, 1).astype('float32')
    test_set=test_set/255.0
    #making predictions 
    model = load_model('/content/drive/MyDrive/Colab Notebooks/JPGS_Final_Model_93_lr0.001.h5')
    loss,accuracy = model.evaluate(test_set,labels)
    print("Test: accuracy = %.2f  ;  loss = %.2f" % (accuracy*100, loss*100))

In [26]:
data_test,labels_test=load_data()
data_closing = preprocess_data(data_test)
number_of_classes = 10
labels_test_encoded = np_utils.to_categorical(labels_test, number_of_classes)

In [27]:
evaluate(data_closing.T,labels_test_encoded)

105/105 [==============================] - 3s 26ms/step - loss: 0.0800 - accuracy: 0.9851
Test: accuracy = 98.51  ;  loss = 8.00


In [33]:
#function to test the model with a hard dataset using a threshold probabibility of 0.7
def hard_evaluation(Input_test): 
    im_siz =30
    test_set = Input_test.reshape(Input_test.shape[0], im_siz, im_siz, 1).astype('float32')
    test_set=test_set/255.0
    #make a prediction
    model = load_model('/content/drive/MyDrive/Colab Notebooks/JPGS_Final_Model_93_lr0.001.h5')
    results = model.predict_proba(test_set)
    #labelling the unknown dataset as -1
    final_results=[]
    for i in range(test_set.shape[0]):
            if (np.amax(results[i])>0.09):
                a=np.argmax(results[i],axis=0)
                final_results=np.append(final_results,a)
            else: 
                final_results=np.append(final_results,-1)
    return final_results

In [35]:
#Code to diaplay the accuracy of our prediction

hard_result=hard_evaluation(data_closing.T)
score=accuracy_score(labels_test,hard_result)
print("Test accuracy for hard test Dataset: %.2f%%" % (score*100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Test accuracy for hard test Dataset: 98.51%
